In [ ]:
import sys
from time import sleep
import spacy
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
def get_geographical_feature(doc):
    for ent in doc.ents:
        if ent.label_ == 'FAC' or ent.label_ == 'GPE' or ent.label_ == 'LOC':
            return ent.text.strip()

In [ ]:
def get_nearest_location(guide_coords, locations):
    best = None
    distance = 0
    
    for location in locations:
        retrieved_coords = (
            round(location.latitude, 2),
            round(location.longitude, 2)
        )
        if not best:
            best = location
            distance = geodesic(guide_coords, retrived_coords).km
        else:
            potential_distance = geodesic(guide_coords, retrieved_coords).km
            if potential_distance <= distance:
                best = location
                distance = potential_distance
                
    return best, distance

In [ ]:
def get_captions(captions):
    nlp = spacy.load("en_core_web_sm")
    Lat = []
    Long = []
    coords = []
#     geolocator = Nominatim(user_agent="coronavirusUK-submission-ner-geocoder")
    geolocator = GoogleV3(scheme='http')
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    for caption in captions:
        feature = get_geographical_feature(nlp(str(caption)))
        location = geolocator.geocode(feature, language="en", limit=1)
        if location:
            # Center Point of the United Kingdom
            guide_coords = (
                round(54.093409, 2), 
                round(-2.89479, 2)
            )
            retrieved_coords = (
                round(location.latitude, 2),
                round(location.longitude, 2)
            )
            distance = geodesic(guide_coords, retrieved_coords).km
            if distance <= 550:
                coords.append(retrieved_coords)
                Lat.append(retrieved_coords[0])
                Long.append(retrieved_coords[1])
            else:
                coords.append(("", ""))
                Lat.append("")
                Long.append("")
                
    return coords
            
    
    